In [ ]:
import sqlite3
import time
import datetime
import numpy as np
import os,glob
import pathlib
from pathlib import Path
import scipy.io.wavfile
import scipy.signal

def sonic_roh_import(Datei, Abtastrate, unterordner=''):
    
    Ordner = pathlib.Path(os.getcwd()).parent.resolve()
    Ordner = Path(Ordner, unterordner)
    filepath = Path(Ordner, Datei)

    file = open(filepath, "rb")  
    raw_data = np.fromfile(file, np.int16)

    # Zeitachse berechnen
    Zeit = np.linspace(0, len(raw_data)/Abtastrate, len(raw_data))# Zeit
    
    return raw_data, Zeit

def sonic_feature_import(Datei, unterordner=''):

    Ordner = os.getcwd()
    Ordner = Ordner[0:-13] + '/' + unterordner # Ordnername '/Python' wird in OP Ordner[0:-7] abgezogen

    dbfile = Ordner + '/' + Datei
    con = sqlite3.connect(dbfile)
    cur = con.cursor()

    cur.execute('SELECT * FROM FeatureTable')

    names = [description[0] for description in cur.description]
    table = [a for a in cur.execute("SELECT name FROM sqlite_master WHERE type = 'table'")]

    cur.execute('SELECT * FROM FeatureTable')
    length = len(cur.fetchall())

    data = np.zeros((len(names), length))
    [a, b] = data.shape 
    date_string = ''
    z = 0

    for line in cur.execute('SELECT * FROM FeatureTable'):# LIMIT 0,100'):
        date_string = line[0]
        unix = time.mktime(datetime.datetime.strptime(date_string, "%Y-%m-%d %H:%M:%S.%f").timetuple())
        data[0,z] = unix
        data[1:,z] = line[1:]
        z += 1

    data[0,:] -= data[0,0]

    con.close()
    
    return data, names